## Loading Packages and Libraries

In [1]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.utils import to_categorical

import pandas as pd
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt 
import os 
import glob
from IPython.display import display, Markdown
import plotly.graph_objs as go
import plotly.offline as py
from scipy.stats import pearsonr
import seaborn as sns
sns.set(style='darkgrid')

## Loading Dataset and Preprocessing

In [2]:
files = os.path.join("ER_dataset/", "*.csv")
files = glob.glob(files)
df = pd.concat(map(pd.read_csv, files))
df.columns = ['# condition','emotion','Feature 1','Feature 2','Feature 3','Feature 4','Feature 5','Feature 6','Feature 7']
from sklearn.model_selection import train_test_split


df['emotion'] = df['emotion'].map({-1:0, 0:1, 1:2})
X = df.drop('emotion', axis=1)
X = df.drop('# condition', axis=1)
y = df.emotion

# Preprocess the data
# Normalize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Encode emotion labels to integers (if not already encoded)
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Convert labels to one-hot encoding
y_onehot = to_categorical(y_encoded)


## Training Process

### Function to create windows

In [3]:

def create_windows(data, labels, window_size, step_size):
    X_windows, y_windows = [], []
    for i in range(0, len(data) - window_size + 1, step_size):
        X_windows.append(data[i:i + window_size])
        y_windows.append(labels[i + window_size - 1])  # Use the last label in the window
    return np.array(X_windows), np.array(y_windows)



###  Buliding LSTM Model

In [6]:
# Model


# Define window size and step size
window_size = 10  # Number of time steps in each window
step_size = 5     # Number of time steps to slide the window

# Create windows for features and labels
X_windows, y_windows = create_windows(X_scaled, y_onehot, window_size, step_size)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_windows, y_windows, test_size=0.2, random_state=42)

model = Sequential()
model.add(LSTM(64, input_shape=(window_size, X_train.shape[2]), return_sequences=False))  # LSTM layer
model.add(Dropout(0.2))  # Dropout for regularization
model.add(Dense(32, activation='relu'))  # Fully connected layer
model.add(Dense(y_onehot.shape[1], activation='softmax'))  # Output layer

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [7]:

# Train the model
history = model.fit(X_train, y_train, epochs=2, batch_size=32, validation_split=0.2, verbose=1)

Epoch 1/2
2596/2596 [==============================] - 37s 13ms/step - loss: 0.0198 - accuracy: 0.9950 - val_loss: 0.0019 - val_accuracy: 0.9995
Epoch 2/2
2596/2596 [==============================] - 31s 12ms/step - loss: 0.0014 - accuracy: 0.9997 - val_loss: 6.2634e-04 - val_accuracy: 0.9999


## Evaluation

In [5]:
# Step 6: Evaluate the model
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_test_classes = np.argmax(y_test, axis=1)

print("Accuracy:", accuracy_score(y_test_classes, y_pred_classes))
print("Classification Report:\n", classification_report(y_test_classes, y_pred_classes))

Accuracy: 0.9999229583975346
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      8630
           1       1.00      1.00      1.00      8685
           2       1.00      1.00      1.00      8645

    accuracy                           1.00     25960
   macro avg       1.00      1.00      1.00     25960
weighted avg       1.00      1.00      1.00     25960

